In [1]:
#embedding 테스트 - 완료
##one-hot으로 할 필요가 없음.
##embedding_size도 마음대로 결정할 수 있음.
##random init으로 해도됨.
##input의 embedding을 training할수도 있고 안할수도 있음
##Y는 embedding 할 필요 없음. 그냥 target class shape만 맞춰서 loss를 구하면 됨

#file에서 읽어와서 input으로 넣어보기 - 완료
##이미 index화 되어 있는 파일 읽어오기 - 완료
##dictionary file로부터 dic 생성 - 완료
##embedding 활용 - 완료

#saver 적용

#padding 적용

#batch 적용

#bidirectional rnn 적용

from __future__ import print_function

import tensorflow as tf
import numpy as np
import pprint as pp
import os

from tensorflow.contrib import rnn
tf.set_random_seed(777)  # reproducibility

In [2]:
checkpoint_path = 'data/dev/ckpt/'

def read_data_from_file(file_path):
    sentence_file = open(file_path, 'r')
    sentences = []
    while True:
        line = sentence_file.readline()
        if not line:
            break;
        sentences.append(line.replace('\n',''))
    sentence_file.close()
    
    rev_vocab_dic = list(set(' '.join(sentences)))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return sentences, vocab_dic, rev_vocab_dic

def read_sentence_indexes_from_file(file_path):
    sentence_index_file = open(file_path, 'r')
    sentence_indexes = []
    while True:
        line = sentence_index_file.readline()
        if not line:
            break;
        sentence_indexes.append([int(c) for c in line.replace('\n','').split()])
    sentence_index_file.close()
    return sentence_indexes

def load_dic(dic_file_path):
    dic_file = open(dic_file_path, 'r')
    rev_vocab_dic = []
    while True:
        line = dic_file.readline()
        if not line:
            break;
        rev_vocab_dic.append(line.replace('\n',''))
    vocab_dic = {w: i for i, w in enumerate(rev_vocab_dic)}
    return vocab_dic, rev_vocab_dic

In [3]:
chosung_sentence_indexes =  read_sentence_indexes_from_file('data/dev/namuwiki_chosung_dev.txt.ids10000')
target_sentence_indexes =  read_sentence_indexes_from_file('data/dev/namuwiki_sentences_dev.txt.ids10000')
chosung_vocab_dic, chosung_rev_vocab_dic = load_dic('data/vocab10000.from')
target_vocab_dic, target_rev_vocab_dic = load_dic('data/vocab10000.to')

In [4]:
data_dim = len(target_vocab_dic)
hidden_size = len(target_vocab_dic)
num_classes = len(target_vocab_dic)
embedding_size = 30
sequence_length = 22  # Any arbitrary number
learning_rate = 0.05

In [5]:
dataX = []
dataY = []
for i in range(len(chosung_sentence_indexes)):
    chosung_indexes = chosung_sentence_indexes[i][:sequence_length]
    target_indexes = target_sentence_indexes[i][:sequence_length]
    dataX.append(chosung_indexes)
    dataY.append(target_indexes)

In [6]:
batch_size = len(dataX)

X = tf.placeholder(tf.int32, [None, sequence_length], name="chosung_sentences")
Y = tf.placeholder(tf.int32, [None, sequence_length], name="target_sentences")

In [7]:
print(num_classes)
embeddings = tf.Variable(
    tf.random_uniform([len(chosung_vocab_dic), embedding_size], -1.0, 1.0), trainable=False, name="chosung_embeddings")
embedded_input = tf.nn.embedding_lookup(embeddings, X, name="embedded_chosung_sentences")
print(embedded_input)

3547
Tensor("embedded_chosung_sentences:0", shape=(?, 22, 30), dtype=float32)


In [8]:
# Make a lstm cell with hidden_size (each unit output vector size)
cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
cell = rnn.MultiRNNCell([cell] * 2, state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(cell, embedded_input, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size], name="reshaped_rnn_output")
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes], name="predictions")

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length], name="seqence_weights")

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights, name="sequence_loss")
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

In [9]:
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state(checkpoint_path)
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print("Reading model parameters from %s" % ckpt.model_checkpoint_path)
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("Created model with fresh parameters.")
    sess.run(tf.global_variables_initializer())

Reading model parameters from data/dev/ckpt/chosung_rnn.ckpt


In [ ]:
for i in range(100):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    if i % 9 == 0:
        print('step:',i,' loss:',l)
        results = sess.run(outputs, feed_dict={X: dataX})
        for sentence_result in results:
            index = np.argmax(sentence_result, axis=1)
            print(''.join([target_rev_vocab_dic[t] for t in index]))
        print('----------')
        checkpoint_file_path = os.path.join(checkpoint_path, "chosung_rnn.ckpt")
        saver.save(sess, checkpoint_file_path)
        print('saved checkpoint')


step: 0  loss: 17.2944
0면해면이면이면이사이면이사이사이사이사이사
0면해면이면이면이사이면이사이사이사이사이사
여러a에서서서서서서서서서서서모모모서하이공
0년들이사삽사이사이사이사이사이사이사이사이
0년예삽사이사이사이사이사이사이사이사이사이
0면해면이면이면이사이면이사이사이사이사이사
0면해면이면이면이사이면이사이사이사이사이사
0면해면이면이면이사이면이사이사이사이사이사
여러룹사삽사삽사이사이사이사이사이사이사이사
벌크서서서서서서서서서서서서서서서서서서서서
----------


In [ ]:
print(len(results)) #batch size
print(len(results[0])) #sequence length
print(len(results[0][0])) #vocab_size

print(batch_size)
print(sequence_length)
print(len(target_vocab_dic))


In [ ]:
#embedding 연습!!
import tensorflow as tf

embeddings = tf.Variable(
    tf.random_uniform([num_classes, 50], -1.0, 1.0))

train_inputs = tf.placeholder(tf.int32, shape=[3,2])
embed = tf.nn.embedding_lookup(embeddings, train_inputs)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
ems, inputs,embd = sess.run([embeddings,train_inputs,embed], feed_dict={train_inputs:[[1,2],[2,3],[3,4]]})

for i in ems:
    print(i)
print('-----------')
print(inputs)
print('-----------')
print(embd)